In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin as pd
from fastai.tabular import *

## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/imps_log'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 清洗掉 ad_static_feature 中没有的广告
- 我们只关心有效广告的曝光情况

In [5]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')
ad_static.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,1552967764,21666,21917,18,198,64
502149,665036,1550740115,6184,11633,18,232,64
502150,491914,1548392470,1809,5278,5,232,64
502151,3990,1551428228,4255,3401,18,218,36
502152,222150,1551340613,4250,14527,18,94,36


In [6]:
ad_static.shape

(502153, 7)

### 广告操作数据

In [7]:
col_names = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值', '人群定向']

In [8]:
ad_operation = pd.read_csv(root/'ad_operation.dat', sep='\t', header=None, names=col_names)

In [9]:
ad_operation.describe()

,广告id,创建/修改时间,操作类型,修改字段,人群定向
count,760866.000000,7.608660e+05,760866.000000,760866.000000,0.0
mean,366502.509025,1.717885e+13,1.147152,1.523331,NaN
std,210931.328143,7.192564e+12,0.354258,0.855417,NaN
min,31.000000,0.000000e+00,1.000000,1.000000,NaN
25%,184651.000000,2.019022e+13,1.000000,1.000000,NaN
50%,363843.000000,2.019030e+13,1.000000,1.000000,NaN
75%,547318.000000,2.019031e+13,1.000000,2.000000,NaN
max,736053.000000,2.019032e+13,2.000000,4.000000,NaN


In [10]:
ad_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760866 entries, 0 to 760865
Data columns (total 6 columns):
广告id       760866 non-null int64
创建/修改时间    760866 non-null int64
操作类型       760866 non-null int64
修改字段       760866 non-null int64
操作后的字段值    760866 non-null object
人群定向       0 non-null float64
dtypes: float64(1), int64(4), object(1)
memory usage: 34.8+ MB


In [11]:
ad_operation.tail(7)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值,人群定向
760859,100409,0,1,1,0,NaN
760860,171392,0,1,1,0,NaN
760861,252704,0,1,1,0,NaN
760862,185679,0,1,1,0,NaN
760863,15132,0,1,1,0,NaN
760864,29322,0,1,1,0,NaN
760865,47325,0,1,1,0,NaN


In [12]:
ad_operation.shape

(760866, 6)

In [22]:
ad_operation.操作类型.unique()

array([2, 1])

 - 合并表格

In [13]:
ads = pd.merge(ad_static, ad_operation, on='广告id', how='inner')

In [14]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值,人群定向
0,347637,1550790222,18529,28775,13,12,34,0,2,2,100,NaN
1,347637,1550790222,18529,28775,13,12,34,0,2,3,"age:217,601,79,202,837,942,638,394,347,731,739...",NaN
2,347637,1550790222,18529,28775,13,12,34,0,2,4,"281474976694272,281474976694272,28147497669427...",NaN
3,347637,1550790222,18529,28775,13,12,34,20190222233702,1,1,0,NaN
4,347637,1550790222,18529,28775,13,12,34,20190223001315,1,2,100,NaN


In [15]:
ads.shape

(615150, 12)

In [16]:
615150 - 502153

112997

In [17]:
760866 - 615150

145716

In [18]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615150 entries, 0 to 615149
Data columns (total 12 columns):
广告id       615150 non-null int64
创建时间       615150 non-null int64
广告账户id     615150 non-null int64
商品id       615150 non-null int64
商品类型       615150 non-null int64
广告行业id     615150 non-null int64
素材尺寸       615150 non-null object
创建/修改时间    615150 non-null int64
操作类型       615150 non-null int64
修改字段       615150 non-null int64
操作后的字段值    615150 non-null object
人群定向       0 non-null float64
dtypes: float64(1), int64(9), object(2)
memory usage: 61.0+ MB


- 数据保存

In [20]:
ads.to_csv('../data/ad_static_operation.csv', index=None, encoding='utf-8')